In [2]:
# Automatically reload modules when code changes
%load_ext autoreload
%autoreload 2

In [3]:
from pgmcp.settings import get_settings

SETTINGS = get_settings()
from rich.table import Table
from rich.console import Console
from rich.theme import Theme


console = Console(highlight=False, force_jupyter=True)

In [4]:
# Mcp Server Imports 
from pgmcp.server_crawl import mcp as mcp_crawl
from pgmcp.server_kb import mcp as mcp_kb


# Ingest Crawl 22 = openwebui.io
# Ingest Crawl 21 = sqlalchemy docs
from fastmcp import Client

async with Client(mcp_kb) as client:
    response = await client.call_tool("ingest_crawl_job", {"crawl_job_id": 41})
    
    print(response)

CallToolResult(content=[TextContent(type='text', text='{"metadata":{"page":1,"per_page":10,"count":0},"record":{"name":"docs.pydantic.dev:  latest ","library_id":1,"id":19,"created_at":"2025-08-12T15:13:47.762001Z","updated_at":"2025-08-12T15:13:47.762001Z","documents_count":111,"chunks_count":3223,"chunks_token_total":965310}}', annotations=None, meta=None)], structured_content={'metadata': {'page': 1, 'per_page': 10, 'count': 0}, 'record': {'name': 'docs.pydantic.dev:  latest ', 'library_id': 1, 'id': 19, 'created_at': '2025-08-12T15:13:47.762001Z', 'updated_at': '2025-08-12T15:13:47.762001Z', 'documents_count': 111, 'chunks_count': 3223, 'chunks_token_total': 965310}}, data={'metadata': {'page': 1, 'per_page': 10, 'count': 0}, 'record': {'name': 'docs.pydantic.dev:  latest ', 'library_id': 1, 'id': 19, 'created_at': '2025-08-12T15:13:47.762001Z', 'updated_at': '2025-08-12T15:13:47.762001Z', 'documents_count': 111, 'chunks_count': 3223, 'chunks_token_total': 965310}}, is_error=False)

In [5]:
from sqlalchemy.future import select
from pgmcp.models import Corpus, Document, Chunk 

table = Table(show_header=True, header_style="bold magenta")
table.add_column("ID")
table.add_column("Title")
table.add_column("Content-Type")
table.add_column("Content-Length", justify="right")

            
async with Corpus.async_context() as session:
    # 4 = sqlalchemy docs
    # 5 = openwebui docs
    if CORPUS := await Corpus.find(19): 
        await CORPUS.ensure_loaded("documents")
        DOCUMENTS = list(CORPUS.documents)

        for i, doc in enumerate(DOCUMENTS):
            if i >= 10:
                table.add_row("...", "...", "...", "...")
                break
            table.add_row(
                str(doc.id), 
                doc.title, 
                doc.content_type, 
                str(len(str(doc.content)))
            )
            
    display(table)
    
    DOCUMENT = DOCUMENTS[0]
    DOCUMENT = await DOCUMENT.ensure_loaded("chunks")

┏━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ ID    ┃ Title                ┃ Content-Type ┃ Content-Length ┃
┡━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ 10581 │ Welcome to Pydantic  │ text/html    │          84232 │
│ 10582 │ Welcome to Pydantic  │ text/html    │          84232 │
│ 10583 │ Models               │ text/html    │         260481 │
│ 10584 │ BaseModel            │ text/html    │         575551 │
│ 10585 │ Architecture         │ text/html    │          82771 │
│ 10586 │ Validating File Data │ text/html    │          83638 │
│ 10587 │ Error Handling       │ text/html    │          97370 │
│ 10588 │ Pydantic Logfire     │ text/html    │          67498 │
│ 10589 │ Pydantic People      │ text/html    │          96532 │
│ 10590 │ Why use Pydantic     │ text/html    │         131638 │
│ ...   │ ...                  │ ...          │            ... │
└───────┴──────────────────────┴──────────────┴────────────────┘

In [6]:
# get all chunks from corpus
import tiktoken
encoder = tiktoken.get_encoding("cl100k_base")

async with Chunk.async_context() as session:
    chunks_qb = Chunk.query().joins(Chunk.document, Document.corpus).where(Corpus.id == 4)
    
    async for chunk in chunks_qb.find_each():
        embeddable_content = chunk.to_embeddable_input()
        if len(encoder.encode(embeddable_content)) > 8192:
            print(f"Chunk {chunk.id} is too long to embed: {len(encoder.encode(embeddable_content))} tokens")
            

In [10]:
# await DOCUMENT.update_embeddings()

In [8]:
import json
from rich.table import Table

table = Table(show_header=True, header_style="bold magenta")
table.add_column("Chunk ID")
# table.add_column("Content")
# table.add_column("Metadata")
table.add_column("chunk_content")
table.add_column("token_count (tiktoken)", justify="right")
table.add_column("token_count (column)", justify="right")



for i, chunk in enumerate(DOCUMENT.chunks):
    if i >= 10:
        break

    token_count_tiktoken = len(encoder.encode(chunk.to_embeddable_input()))
    token_count_column = chunk.token_count

    if token_count_tiktoken != token_count_column:
        table.add_row(str(i), chunk.to_embeddable_input(), str(token_count_tiktoken), str(token_count_column))


console.print(table)

┏━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Chunk ID ┃ chunk_content ┃ token_count (tiktoken) ┃ token_count (column) ┃
┡━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
└──────────┴───────────────┴────────────────────────┴──────────────────────┘

In [ ]:
from pgmcp.models import Corpus
import asyncio

async def embed_corpus(corpus_id=5):
    corpus = await Corpus.find(corpus_id)
    if corpus:
        print(f"Embedding all chunks for corpus: {corpus.name} (ID: {corpus_id})")
        await corpus.update_embeddings()
        print("Embedding complete.")
    else:
        print(f"Corpus with ID {corpus_id} not found.")

await embed_corpus(5)